# 推論・提出 テンプレート

このノートブックは、Kaggle競技における推論と提出ファイル作成のテンプレートです。

## 1. ライブラリのインポート

In [ ]:
import pandas as pd
import numpy as np
import joblib
import os
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

# 設定
pd.set_option("display.max_columns", None)

## 2. データとモデルの読み込み

In [ ]:
# テストデータの読み込み
X_test = pd.read_csv("../data/processed/X_test_processed.csv")
print(f"テストデータ形状: {X_test.shape}")

# 元のテストデータ（IDカラム用）
test_original = pd.read_csv("../data/raw/test.csv")
print(f"元のテストデータ形状: {test_original.shape}")

# モデルの読み込み（例：joblib形式）
model_dir = "../models/"
if os.path.exists(model_dir):
    model_files = [f for f in os.listdir(model_dir) if f.endswith(".joblib")]
    print(f"利用可能なモデル: {model_files}")
    
    # 複数のモデルを読み込み（アンサンブル用）
    models = {}
    for model_file in model_files:
        model_name = model_file.replace(".joblib", "")
        models[model_name] = joblib.load(os.path.join(model_dir, model_file))
        print(f"モデル読み込み完了: {model_name}")
else:
    print("モデルディレクトリが見つかりません")

## 3. 推論実行

In [ ]:
# 推論実行（モデルが読み込まれている場合）
if "models" in locals() and models:
    if len(models) == 1:
        # 単一モデルでの推論
        model_name = list(models.keys())[0]
        model = models[model_name]
        
        print(f"単一モデル推論: {model_name}")
        predictions = model.predict(X_test)
        
    else:
        # 複数モデルのアンサンブル
        print("アンサンブル推論")
        all_predictions = []
        
        for model_name, model in models.items():
            pred = model.predict(X_test)
            all_predictions.append(pred)
            print(f"{model_name}の推論完了")
        
        # 平均アンサンブル
        predictions = np.mean(all_predictions, axis=0)
        print("アンサンブル完了（平均）")
    
    print(f"予測値の形状: {predictions.shape}")
    print(f"予測値の統計: min={predictions.min():.6f}, max={predictions.max():.6f}, mean={predictions.mean():.6f}")
else:
    print("モデルが読み込まれていません。ダミーの予測値を作成します。")
    predictions = np.random.random(len(X_test))

## 4. 提出ファイルの作成

In [ ]:
# IDカラムの特定（適宜調整）
id_column = "id"  # 競技に応じて変更
target_column = "target"  # 競技に応じて変更

# 提出ファイルの作成
if id_column in test_original.columns:
    submission = pd.DataFrame({
        id_column: test_original[id_column],
        target_column: predictions
    })
else:
    # IDカラムがない場合は連番で作成
    submission = pd.DataFrame({
        id_column: range(len(predictions)),
        target_column: predictions
    })

print(f"提出ファイル形状: {submission.shape}")
print("提出ファイルの最初の5行:")
display(submission.head())

print("\n提出ファイルの最後の5行:")
display(submission.tail())

## 5. 提出ファイルの保存

In [ ]:
# タイムスタンプ付きファイル名
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"submission_{timestamp}.csv"
filepath = f"../submissions/{filename}"

# ディレクトリが存在しない場合は作成
os.makedirs("../submissions", exist_ok=True)

# ファイル保存
submission.to_csv(filepath, index=False)
print(f"提出ファイルを保存しました: {filepath}")

# 最新の提出ファイルとしてもコピー
latest_filepath = "../submissions/latest_submission.csv"
submission.to_csv(latest_filepath, index=False)
print(f"最新提出ファイルを保存しました: {latest_filepath}")